In [1]:
import pandas as pd
import numpy as np
import html
import nltk
import english_words
from nltk.corpus import words
import matplotlib.pyplot as plt
import demoji
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [2]:
dane=pd.read_csv("George.csv")
dane=dane.rename(columns={"0": "Title",'1':"Author", "2":"Comment"})
dane=dane.drop(columns="Unnamed: 0")

def punc_clean(text): # removing special characters
    import string as st
    a=[w for w in text if w not in st.punctuation]
    return ''.join(a)


        

def remove_stopword(text): # removing words which do not affect to sentiment like 'the'
    stopword=nltk.corpus.stopwords.words('english')
    stopword.append("")
    stopword.remove('not') #removing 'not'
    a=[w for w in nltk.word_tokenize(text) if w not in stopword]
    return ' '.join(a)

lista2=[]
for sentence in dane.Comment: # removing html charts like &#39; instead of " ' ".
    sentence=html.unescape(sentence)
    lista2.append(sentence)
dane["review"]=lista2
dane['review'] = dane['review'].apply(remove_stopword)
dane['review'] = dane['review'].apply(punc_clean)
dane.review=dane.review.str.lower() # low case 
lista4=[]
for i in dane.review:
    lista4.append(demoji.replace(i,"")) # removing emojis
dane['review']=lista4
data=pd.read_csv('train.tsv', sep='\t') # database for training
data["Sentiment"]=data["Sentiment"]+1 # scale 1-5
vectorizer = TfidfVectorizer(ngram_range=(1,2),min_df=1) # ngram_range=(1,2) one or two words next to each other
X_train= vectorizer.fit_transform(data['Phrase'])
y_train=data["Sentiment"]
X_test= vectorizer.transform(dane['review'])
classifier = MultinomialNB() # using of Naive Bayes classifier
classifier.fit(X=X_train, y=y_train)

predicted= classifier.predict(X_test)
dane["wynik"]=predicted


In [4]:
dane_testowe=dane[dane.wynik<3] # amount of potential haters and their names
pot_haterzy=dane_testowe.Author.value_counts()
haterzy=pot_haterzy[pot_haterzy.values>3]

In [5]:
# amount of spamers
dane_testowe1=dane.set_index("Author") 
dane_testowe2=dane_testowe1.Comment.value_counts()
pot_spam=dane_testowe2[dane_testowe2.values>15]
spam_list=pot_spam.index.to_list()
listaa=[]
for i in range(len(spam_list)):
    listaa.append(dane_testowe1[dane_testowe1.Comment==spam_list[i]])
user_spam=[] # list of spamers
for j in range(len(listaa)):
    user_spam.append(listaa[j].index[0])


Testing of model #1

In [6]:
dane2=pd.read_csv("George-31.10.csv")
dane2=dane2.rename(columns={"0": "Title",'1':"Author", "2":"Comment"})
dane2=dane2.drop(columns="Unnamed: 0")
dane2=dane2.iloc[:78]
def punc_clean(text): 
    import string as st
    a=[w for w in text if w not in st.punctuation]
    return ''.join(a)


        

def remove_stopword(text): 
    stopword=nltk.corpus.stopwords.words('english')
    stopword.append("")
    stopword.remove('not')
    a=[w for w in nltk.word_tokenize(text) if w not in stopword]
    return ' '.join(a)

lista5=[]
for sentence3 in dane2.Comment:
    sentence3=html.unescape(sentence3)
    lista5.append(sentence3)
dane2["review"]=lista5
dane2['review'] = dane2['review'].apply(remove_stopword)
dane2['review'] = dane2['review'].apply(punc_clean)
dane2.review=dane2.review.str.lower()
lista6=[]
for i in dane2.review:
    lista6.append(demoji.replace(i,""))
dane2['review']=lista6

In [9]:
tf_test2=vectorizer.transform(dane2.review)
predicted2= classifier.predict(tf_test2)
dane2["score"]=predicted2

In [10]:
dane2[dane2.score==4]

,Title,Author,Comment,review,wynik,score
2,Whitney Webb (Global Elite Deep Dive...What Is...,kyle petan,Nice,nice,4,4
18,Whitney Webb (Global Elite Deep Dive...What Is...,A,Wow you have Witney on great,wow witney great,4,4


In [11]:
dane2.score.value_counts()

3    76
4     2
Name: score, dtype: int64